In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import numpy as np

In [ ]:
sns.set_theme()

In [ ]:
df_ba_beers     = pd.read_csv('../data/beer_advocate/beers.csv')
df_ba_breweries = pd.read_csv('../data/beer_advocate/breweries.csv')
df_ba_users     = pd.read_csv('../data/beer_advocate/users.csv')
df_ba_ratings   = pd.read_csv('../data/beer_advocate/ratings.csv')

# Beer Advocate - Trends Analysis (Question 5)

## Data Exploration

In [ ]:
df_ba_ratings['date_week'] = pd.to_datetime(df_ba_ratings['date'], unit="s").dt.to_period('W')
df_ba_ratings['date_day']  = pd.to_datetime(df_ba_ratings['date'], unit="s").dt.round('D')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
df_ba_ratings.groupby('date_week').size().plot(
    title='Number of ratings per week',
    xlabel='Week',
    ylabel='Number of ratings',
    ax=axes[0]
)
df_ba_ratings[
    (df_ba_ratings['date_week'] >= '2011-01-01') &
    (df_ba_ratings['date_week'] <= '2013-01-01')
].groupby('date_week').size().plot(
    title='Number of ratings per week',
    xlabel='Week',
    ylabel='Number of ratings',
    ax=axes[1]
)
plt.show()

It seems that something happended around November 2011. Indeed, there is a period spanning approximately 45 days with a very high number of ratings compared to usual regime. After performing some research we were not able to find the cause of this peak. We decided to focus on the data covering the period after November 2011 since it represents ~80% of our data.

In [ ]:
CUTOFF_DATE_START = '2011-11-14'
CUTOFF_DATE_END = '2012-01-01'

print((df_ba_ratings['date_day'] >= CUTOFF_DATE_END).mean())

stats.ttest_ind(
    df_ba_ratings[df_ba_ratings['date_day'] < CUTOFF_DATE_START]['rating'],
    df_ba_ratings[df_ba_ratings['date_day'] >= CUTOFF_DATE_END]['rating']
)

In [ ]:
df_ba_ratings_trend = df_ba_ratings[df_ba_ratings['date_day'] >= CUTOFF_DATE_END].copy()

In [ ]:
df_ba_ratings_trend.groupby('date_week').size().plot(
    title='Number of ratings per week',
    xlabel='Week',
    ylabel='Number of ratings'
)
plt.show()

## Time series analysis

In [ ]:
# Dataframe covering all days on the analysed period
df_dates = pd.DataFrame(index=pd.date_range(
    start=pd.to_datetime(CUTOFF_DATE_END).round('D'),
    end=pd.to_datetime(df_ba_ratings_trend['date'].max(), unit='s').round('D'),
freq='D'))

In [ ]:
# Dataframe containing the number of ratings per day for each style (with missing days)
df_partial_time_series = df_ba_ratings_trend.groupby(['date_day', 'beer_global_style'])\
    .size()\
    .reset_index(level=1, name='count')\
    .pivot(columns='beer_global_style', values='count')

In [ ]:
# Merging and filling NA with 0 in order to have full time series for each style
df_time_series = pd.merge(
    df_dates,
    df_partial_time_series,
    how='left', left_index=True, right_index=True
).fillna(0)

### Decomposition

Using the moving averages and assuming that the observed time serie $O$ can be decomposed in an additive way in $O = T + S + R$ where $T$ is the general trend, $S$ the seasonal effect and $R$ the residuals effects, we can split the time serie into those 3 components.

The residuals $R$ are some sense the effect that cannot be explained by seasonality or general trend (e.g. popularity of BeerAdvocate, increasing beer market, etc.) and thus are interesting to detect trends.

In [ ]:
seasonal_decompose(df_time_series['India Pale Ale'], model='additive', period=365).plot()
plt.show()

In [ ]:
df_residuals = df_time_series.apply(lambda ts: seasonal_decompose(ts, model='additive', period=365).resid).dropna()

The dataframe `df_residuals` contains, for each beer style, the time serie of the residuals. In order to compare the residuals of different styles, we need to normalize the time series to compare meaningful values (Z-scores).

In [ ]:
df_residuals = (df_residuals - df_residuals.mean()) / df_residuals.std()

In [ ]:
df_residuals.head(5)

### Trends identification - Approach 1

Approach 1 is based on two criteria computed on a moving window :

- **Intra variation** : Compute the Z-scores on the time serie for a given style
- **Inter variation** : Compute the Z-scores of all styles on a given time period (window)

The interpretation is that a high **intra** Z-score means that this style has a significantly high number of ratings compared to the usual number of ratings of this style. A high **inter** Z-score means that this style has a significantly high number of ratings compared to other styles during this time period.

In [ ]:
def plot_hype_period(
        df_residuals: pd.DataFrame, 
        style: str,
        window_size: int = 14,
        intra_threshold: float = 2,
        inter_threshold: float = 2
    ):

    df_residuals_intra = (df_residuals.rolling(window=window_size).mean() - df_residuals.mean()) / df_residuals.std().dropna()
    df_residuals_inter = df_residuals.rolling(window=window_size).mean().dropna().apply(stats.zscore, axis=1)

    serie = df_residuals_intra[style]
    values = serie[(df_residuals_intra[style] > intra_threshold) & (df_residuals_inter[style] > inter_threshold)]

    plt.figure(figsize=(12, 4))
    plt.title(f'[{style}] Normalized residuals (rolling average of {window_size} days) (intra z-score > {intra_threshold}, inter z-score > {inter_threshold})')
    plt.xlabel('Date')
    plt.ylabel('Normalized residuals')
    plt.plot(serie, alpha=0.5)
    plt.scatter(values.index, values, color='red', label='Hype period')
    plt.legend()
    plt.show()

In [ ]:
plot_hype_period(df_residuals, 'India Pale Ale', window_size=7, intra_threshold=1.5, inter_threshold=1.5)

Interpretation of each red dot (hype) :
- **Intra** : the average number of ratings of the past 7 days was  1.5 standard deviation higher than the average number of ratings for this style during the entire timeframe of the dataset
- **Inter** : the average number of ratings of the past 7 days was 1.5 standard deviation higher than the average number of ratings for all style during those 7 days

### Trends Identification - Approach 2

Approach 2 keeps the criteria of intra and inter variation but rather than looking at moving averages, it analyze the number of days during which the constraint is respected. For example for the intra criteria, the Z-score should be higher than the threshold each day of the window (rather than the mean of the window as in approach 1). Approach 2 is therefore more strict than approach 1, but is also more sensitive to noise (e.g. hype period of 14 days, if one day, there is less rating, it will be missed) 

In [ ]:
def plot_hype_period_2(
        df_residuals: pd.DataFrame, 
        style: str,
        window_size: int = 3,
        intra_threshold: float = 2,
        inter_threshold: float = 2
    ):
    serie = df_residuals[style]

    df_residuals_intra = (serie - serie.mean()) / serie.std()
    df_residuals_inter = df_residuals.apply(stats.zscore, axis=1)[style]

    intra_condition = (df_residuals_intra > intra_threshold).rolling(window=window_size).apply(lambda x: x.all(), raw=True)
    inter_condition = (df_residuals_inter > inter_threshold).rolling(window=window_size).apply(lambda x: x.all(), raw=True)

    values = serie.where((intra_condition == 1) & (inter_condition == 1))

    plt.figure(figsize=(12, 4))
    plt.title(f'[{style}] Normalized residuals (constraint window of {window_size} days) (intra z-score > {intra_threshold}, inter z-score > {inter_threshold})')
    plt.xlabel('Date')
    plt.ylabel('Normalized residuals')
    plt.plot(serie, alpha=0.5)

    plt.scatter(values.index, values, color='red', label='Hype period')
    plt.legend()
    plt.show()

In [ ]:
plot_hype_period_2(df_residuals, 'Dark Lager', window_size=3, intra_threshold=1.5, inter_threshold=1.5)

Interpretation : intra and inter z-scores are greather than the threshold for 3 consecutive days.

### Trends Identification - Approach 3 

In [ ]:
def plot_hype_period_3(
        df_residuals: pd.DataFrame, 
        style: str,
        window_size_pct_change: int = 3,
        window_size_constraint: int = 14
    ):
    serie = df_residuals[style]

    smoothed_changes = serie.pct_change().rolling(window=window_size_pct_change, center=True).mean()

    changes_threshold = (0 - smoothed_changes.mean()) / smoothed_changes.std()
    smoothed_changes = (smoothed_changes - smoothed_changes.mean()) / smoothed_changes.std()

    changes_condition = (smoothed_changes > changes_threshold).rolling(window=window_size_constraint).apply(lambda x: x.all(), raw=True)
    values = serie.where(changes_condition == 1)

    plt.figure(figsize=(12, 4))
    plt.title(f'[{style}] Normalized residuals (constraint window of {window_size_constraint} days) (changes computed on {window_size_pct_change} days)')
    plt.xlabel('Date')
    plt.ylabel('Normalized residuals')

    plt.plot(serie, alpha=0.5)
    plt.plot(smoothed_changes, alpha=0.5)
    plt.scatter(values.index, values, color='red', label='Hype period')
    plt.legend()
    plt.show()

In [ ]:
plot_hype_period_3(df_residuals, 'India Pale Ale', window_size_pct_change=5, window_size_constraint=14)

Interpretation : percentage change of the number of ratings average on a 5 days sliding windows is positive (increasing number of ratings) for 14 consecutives days.

# Sentiment Analysis (Question 4)

In [ ]:
HUGGING_FACE_MODEL = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(HUGGING_FACE_MODEL)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def predict_rating(reviews: pd.Series) -> pd.DataFrame:
    encoded_input = tokenizer(reviews.tolist(), padding=True, truncation=True, max_length=512, return_tensors='pt')
    encoded_input = {key: tensor.to(device) for key, tensor in encoded_input.items()}
    with torch.no_grad():
        output = model(**encoded_input)
    scores = softmax(output.logits, dim=1)
    return pd.DataFrame([
        *scores.cpu().numpy()
    ], columns=['1', '2', '3', '4', '5'], index=reviews.index)

In [ ]:
batch = df_ba_ratings[df_ba_ratings['text'].str.len() > 377]['text'].iloc[:100]

In [ ]:
df_predictions = predict_rating(batch)